In [71]:
# %pip install tensorflow==2.12.0 scikit-learn numpy pandas==1.5.2 rich==13.3.3 matplotlib


In [72]:
# %pip install ta==0.10.2 yfinance==0.2.13

In [73]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import talib
import talib as ta
import tensorflow as tf
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from ta import add_all_ta_features
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

In [74]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [75]:
# Load the Bitcoin price data
bitcoin_data = pd.read_csv(r'C:\Users\MrBios\Documents\Development\test\csv\bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')  # Replace "bitcoin_data.csv" with your actual data file
bitcoin_data.dropna(inplace=True)

In [76]:
# Calculate technical indicators using TA-Lib
bitcoin_data["RSI"] = talib.RSI(bitcoin_data["Close"])
bitcoin_data["MACD"], bitcoin_data["MACD_SIGNAL"], _ = talib.MACD(bitcoin_data["Close"])

In [77]:
# remove timestamp column
bitcoin_data.drop(columns=["Timestamp"], inplace=True)

In [78]:
# Prepare the data for training
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(bitcoin_data[["Close", "RSI", "MACD", "MACD_SIGNAL"]])


In [79]:

# Create sequences of data with a given time step
time_step = 60  # Number of previous time steps to consider for predicting the next time step
X = []
y = []
for i in range(time_step, len(scaled_data)):
    X.append(scaled_data[i - time_step:i, 1:])
    y.append(scaled_data[i, 0])
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)